In [1]:
import pandas as pd
import torch
import numpy as np

In [2]:
# Loading the dataset
data1 = pd.read_csv('MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv')
#data2 = pd.read_csv('MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv')
#data3 = pd.read_csv('MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv')
data4 = pd.read_csv('MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
#data5 = pd.read_csv('MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
#data6 = pd.read_csv('MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv')
data7 = pd.read_csv('MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
#data8 = pd.read_csv('MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [3]:
data_list = [data1,data4,data7]

print('Data dimensions: ')
for i, data in enumerate(data_list, start = 1):
  rows, cols = data.shape
  print(f'Data{i} -> {rows} rows, {cols} columns')

Data dimensions: 
Data1 -> 529918 rows, 79 columns
Data2 -> 170366 rows, 79 columns
Data3 -> 286467 rows, 79 columns


In [4]:
df = pd.concat(data_list)
rows, cols = df.shape

print('New dimension:')
print(f'Number of rows: {rows}')
print(f'Number of columns: {cols}')
print(f'Total cells: {rows * cols}')

New dimension:
Number of rows: 986751
Number of columns: 79
Total cells: 77953329


In [5]:
# Deleting dataframes after concating to save memory
for d in data_list: del d

In [6]:
# Renaming the columns by removing leading/trailing whitespace
col_names = {col: col.strip() for col in df.columns}
df.rename(columns = col_names, inplace = True)

In [7]:
print(df.columns)

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

In [8]:
# Separate features from labels
X = df.drop(columns=['Label'])
y = df['Label']

In [9]:
# Replace ±∞ with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# Suppose the dataset has a 'Label' column
df['Label'] = df['Label'].astype('category').cat.codes

# Separate features & labels
X = df.drop(columns=['Label'])
y = df['Label'].values

print("Features shape:", X.shape)
print("Unique labels in y:", np.unique(y))

Features shape: (985808, 78)
Unique labels in y: [0 1 2 3 4]


In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_scaled, y, test_size=0.15, random_state=42, stratify=y
)

val_ratio = 0.15 / 0.85
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=val_ratio, 
    random_state=42, stratify=y_train_val
)

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)


Train: (690064, 78) (690064,)
Val:   (147872, 78) (147872,)
Test:  (147872, 78) (147872,)


Justification for 70-15-15 split:

70% Training: Enough data to learn robust patterns.

15% Validation: A separate set for hyperparameter tuning (e.g., learning rate, noise multiplier for DP, etc.).

15% Test: Kept strictly for final evaluation. This prevents overfitting to the validation set and provides an unbiased measure of performance.

In [11]:
#%pip install torch opacus

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from opacus import PrivacyEngine

In [13]:
# Convert to PyTorch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)

X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.long)

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset   = TensorDataset(X_val_t,   y_val_t)
test_dataset  = TensorDataset(X_test_t,  y_test_t)

batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)


In [14]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )
    def forward(self, x):
        return self.net(x)

input_dim = X_train.shape[1]
num_classes = len(np.unique(y_train))  # If it's binary, likely 2
model = SimpleMLP(input_dim, hidden_dim=64, num_classes=num_classes)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# DP hyperparameters
noise_multiplier = 1.0
max_grad_norm = 1.0
epochs = 5
delta = 1e-5

privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=noise_multiplier,
    max_grad_norm=max_grad_norm,
)


c:\Users\poly\AppData\Local\Programs\Python\Python313\Lib\site-packages\opacus\privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


In [16]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * data.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == target).sum().item()
        total += target.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc  = correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")


c:\Users\poly\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Epoch [1/5], Loss: 0.2455, Acc: 0.9184
Epoch [2/5], Loss: 0.0474, Acc: 0.9907
Epoch [3/5], Loss: 0.0496, Acc: 0.9919
Epoch [4/5], Loss: 0.0499, Acc: 0.9923
Epoch [5/5], Loss: 0.0495, Acc: 0.9924


In [17]:
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
)

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    all_probs = []
    all_labels = []
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            
            # If multi-class, outputs have shape [batch_size, num_classes]
            probs = torch.softmax(outputs, dim=1)[:,1]  # Probability of class=1
            all_probs.append(probs.cpu().numpy())
            all_labels.append(target.cpu().numpy())
            
            _, pred = torch.max(outputs, 1)
            correct += (pred == target).sum().item()
            total += target.size(0)
    acc = correct / total
    return acc, np.concatenate(all_probs), np.concatenate(all_labels)

test_acc, y_prob_test, y_test_true = evaluate(model, test_loader)
print("Test Accuracy:", test_acc)

y_pred_test = (y_prob_test >= 0.5).astype(int)

prec = precision_score(y_test_true, y_pred_test,average='weighted')
rec  = recall_score(y_test_true, y_pred_test,average='weighted')
f1   = f1_score(y_test_true, y_pred_test,average='weighted')
cm   = confusion_matrix(y_test_true, y_pred_test)

print("Precision:", prec)
print("Recall:", rec)
print("F1-Score:", f1)
print("Confusion Matrix:\n", cm)

Test Accuracy: 0.9920945141744211
Precision: 0.9899418263051223
Recall: 0.9920877515689245
F1-Score: 0.9910042453351676
Confusion Matrix:
 [[123107    617      0      0      0]
 [   226  23595      0      0      0]
 [   226      0      0      0      0]
 [     3      0      0      0      0]
 [    98      0      0      0      0]]


c:\Users\poly\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
epsilon = privacy_engine.get_epsilon(delta=delta)
print(f"ε = {epsilon:.2f} for δ = {delta}")

ε = 0.20 for δ = 1e-05


In [ ]:
#%pip install flwr opacus

  Using cached protobuf-4.25.6-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/532.1 kB ? eta -:--:--
   ---------------------------------------- 532.1/532.1 kB 9.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ----------------------------------- ---- 2.9/3.2 MB 14.5 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 12.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ------------------------ --------------- 2.6/4.3 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 11.6 MB/s eta 0:00:00
Using c

In [ ]:
import numpy as np
import flwr as fl
import torch
from torch.utils.data import DataLoader, TensorDataset
from opacus import PrivacyEngine
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch.nn as nn
import torch.optim as optim

# Define the model
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# Define the Flower client
class FLClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader, device):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.01)

        # Initialize Opacus
        self.privacy_engine = PrivacyEngine()

        # Make the model and optimizer "private"
        self.model, self.optimizer, self.train_loader = self.privacy_engine.make_private(
            module=self.model,
            optimizer=self.optimizer,
            data_loader=self.train_loader,
            noise_multiplier=1.0,
            max_grad_norm=1.0,
        )

    def get_parameters(self):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for _ in range(1):  # One epoch
            for data, target in self.train_loader:
                data, target = data.to(self.device), target.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(data)
                loss = self.criterion(output, target)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters(), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        loss = 0.0
        correct = 0
        with torch.no_grad():
            for data, target in self.val_loader:
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                loss += self.criterion(output, target).item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        loss /= len(self.val_loader.dataset)
        accuracy = correct / len(self.val_loader.dataset)
        return float(loss), len(self.val_loader.dataset), {"accuracy": float(accuracy)}

# Example data loading
def load_data():
    """
    Replace this dummy data logic with your actual data loading and preprocessing.
    X_train, y_train, X_val, y_val should be numpy arrays (or easily convertible to tensors).
    """
    # For demonstration, generate a small random dataset
    np.random.seed(42)
    X_train = np.random.randn(1000, 10)
    y_train = np.random.randint(0, 2, size=(1000,))
    X_val = np.random.randn(200, 10)
    y_val = np.random.randint(0, 2, size=(200,))

    X_train_t = torch.tensor(X_train, dtype=torch.float32)
    y_train_t = torch.tensor(y_train, dtype=torch.long)
    X_val_t = torch.tensor(X_val, dtype=torch.float32)
    y_val_t = torch.tensor(y_val, dtype=torch.long)

    train_dataset = TensorDataset(X_train_t, y_train_t)
    val_dataset = TensorDataset(X_val_t, y_val_t)

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

    return train_loader, val_loader

# Federated learning setup
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Example data
    train_loader, val_loader = load_data()

    # Dynamically determine input and output dimensions from the data
    input_dim = next(iter(train_loader))[0].shape[1]  # number of features
    num_classes = 2  # For binary classification, adjust if needed

    model = SimpleMLP(input_dim=input_dim, hidden_dim=64, num_classes=num_classes).to(device)

    client = FLClient(model, train_loader, val_loader, device)
    fl.client.start_client(
    server_address="localhost:8080",
    client=client.to_client()
)

if __name__ == "__main__":
    main()


	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
03/12/2025 17:37:44:WARNING:DEPRECATED FEATURE: flwr.client.start_numpy_client() is deprecated. 
	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, u

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)", grpc_status:14, created_time:"2025-03-12T21:37:46.8568111+00:00"}"
>